In [1]:
import os
import numpy as np
from PIL import Image
from torch.utils.data import Dataset
import torch
import torch.nn as nn
from torchvision import transforms,models
from torch.utils.data import DataLoader
import glob
from sklearn.model_selection import train_test_split

In [2]:
# Jupyter notebook: https://www.kaggle.com/code/xdnuos/drowsiness
# Dataset: https://www.kaggle.com/datasets/xdnuos/drowsiness-dataset

In [3]:
images=glob.glob(os.path.join('./train/dataset/','*.jpg'))

Tiền xử lý

In [4]:
train_list, test_list=train_test_split(images,test_size=.2,shuffle=True)

train_transform = transforms.Compose([
        transforms.RandomHorizontalFlip(p=0.25),
        transforms.RandomRotation(60),
        transforms.Resize(145),
        transforms.ToTensor(), # Converting Tensor - Processable Data
        transforms.Normalize((0, 0, 0),(1, 1, 1))
    ])

val_transform = transforms.Compose([ 
        transforms.ToTensor(),
        transforms.Normalize((0, 0, 0),(1, 1, 1))
    ])

In [5]:
class OpenClosedEyeSet(Dataset):

    def __init__(self,images_list,mode='train',transform=None) -> None:
    
        self.label=None
        self.images_list=images_list
        self.mode=mode
        self.transform=transform
    
    # DUNDER - OVERRIDE
    def __len__(self):
    
        self.dataset_len=len(self.images_list)
        return self.dataset_len

    def __getitem__(self, index):

        image_name=self.images_list[index]
        
        image=Image.open(image_name).convert('RGB')

        # Resizing image to 100 width , 100 height
        image=image.resize((145,145))

        # Whole defined transformations apply on image HERE.
        transformed_image = self.transform(image)

        # Split the labels from file name.
        label=image_name.split('/')[-1].split(".")[0]

        if self.mode=='train' or self.mode=='test':
            if label=='open':
                self.label=0
            elif label=='closed':
                self.label=1

            return transformed_image,self.label

In [6]:
batch_size=40
num_epochs=50

DATASET

In [7]:
train_dataset=OpenClosedEyeSet(train_list,mode='train',transform=train_transform)
test_dataset=OpenClosedEyeSet(test_list,mode='test',transform=val_transform)

DATALOADER

In [8]:
train_dataloader=DataLoader(train_dataset,batch_size=batch_size, shuffle=True,drop_last=True)
val_dataloader=DataLoader(test_dataset,batch_size=batch_size,shuffle=True,drop_last=True)

In [9]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


Network

In [10]:
use_pretrained = True
learning_rate=0.01

In [11]:
# # GOOGLENET
# model = models.googlenet(pretrained=True)
# # print(model)
# # model.fc = nn.Linear(in_features=1024, out_features=2)
# model.classifier = nn.Linear(in_features=4096, out_features=2)
# optimizer = torch.optim.SGD(params=model.parameters(), lr=learning_rate, momentum=0.9)

In [12]:
# #Mobile net
# model = models.mobilenet_v3_large(use_pretrained)
# model.classifier[3] = nn.Linear(in_features=1280, out_features=2)
# # print(model)
# params_to_update = []

# update_params_name = ["classifier.3.weight", "classifier.3.bias"]

# for name, param in model.named_parameters():
#     if name in update_params_name:
#         param.requires_grad = True
#         params_to_update.append(param)
#         print(name)
#     else:
#         param.requires_grad = False
# optimizer = torch.optim.SGD(params=params_to_update, lr=learning_rate, momentum=0.9)

In [13]:
#Alexnet
model  = models.alexnet(use_pretrained)
# print(model)
model.classifier[6] = nn.Linear(in_features=4096, out_features=2)

params_to_update = []

update_params_name = ["classifier.6.weight", "classifier.6.bias"]

for name, param in model.named_parameters():
    if name in update_params_name:
        param.requires_grad = True
        params_to_update.append(param)
        print(name)
    else:
        param.requires_grad = False
optimizer = torch.optim.SGD(params=params_to_update, lr=learning_rate, momentum=0.9)

Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to /root/.cache/torch/hub/checkpoints/alexnet-owt-7be5be79.pth


  0%|          | 0.00/233M [00:00<?, ?B/s]

classifier.6.weight
classifier.6.bias


In [14]:
# # densenet201
# model = models.densenet201(use_pretrained)
# model.classifier = nn.Linear(in_features=1920, out_features=2)
# print(model)
# optimizer = torch.optim.SGD(params=model.parameters(), lr=learning_rate, momentum=0.9)

In [15]:
# # wide_resnet101_2
# model = models.wide_resnet101_2(use_pretrained)
# # print(model)
# model.fc = nn.Linear(in_features=2048, out_features=2)
# optimizer = torch.optim.SGD(params=model.parameters(), lr=learning_rate, momentum=0.9)

In [16]:
# # VGG16
# model = models.vgg16(use_pretrained)
# model.classifier[6] = nn.Linear(in_features=4096, out_features=2)
# params_to_update = []

# update_params_name = ["classifier.6.weight", "classifier.6.bias"]

# for name, param in model.named_parameters():
#     if name in update_params_name:
#         param.requires_grad = True
#         params_to_update.append(param)
#         print(name)
#     else:
#         param.requires_grad = False

# print(params_to_update)
# optimizer = torch.optim.SGD(params=params_to_update, lr=learning_rate, momentum=0.9)

Loss and optimize

In [17]:
# setting mode
model = model.train()
# criterion=nn.BCEWithLogitsLoss()
# optimizer=torch.optim.Adam(params=model.parameters(),lr=learning_rate)
criterion=nn.CrossEntropyLoss()

In [18]:
from tqdm import tqdm

train_losses = []
val_losses = []

train_acc = []
val_acc = []

model.to(device)

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

Train and test

In [19]:
for epoch in range(num_epochs):
    
    model.train()
    running_loss = 0
    for images, labels in tqdm(train_dataloader):
        
        # load to gpu
        images = images.to(device)
        labels = labels.to(device)
        # labels=labels.unsqueeze(1)
        # labels=labels.float()

        # forward pass
        outputs = model(images)

        loss = criterion(outputs, labels)
        running_loss += loss.item()
        
        # backprop and update model params
        # zero the gradient descent
        optimizer.zero_grad()
        # back prop
        loss.backward()
        # after zero processing make optimizer ready
        optimizer.step()

        _, train_preds = torch.max(outputs, 1)
        train_preds  = train_preds.tolist()
        train_acc_item = [1 if train_preds[i] == labels[i] else 0 for i in range(len(outputs))]
        
    #Summing over all correct predictions
    train_acc.append((np.sum(train_acc_item) / len(train_preds))*100)
    # calculate training loss for the epoch
    train_losses.append(running_loss / len(train_dataloader))
    # train_losses.append(loss)
    
    # calculate loss accuracy on validation set
    model.eval()
    running_loss = 0
    
    with torch.no_grad():  
        for images, labels in tqdm(val_dataloader):
            
            # load to gpu
            images = images.to(device)
            labels = labels.to(device)
            
            # forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)
            running_loss += loss.item()

            _, preds = torch.max(outputs, 1)
            preds  = preds.tolist()
            acc = [1 if preds[i] == labels[i] else 0 for i in range(len(outputs))]

    
    #Summing over all correct predictions
    val_acc.append((np.sum(acc) / len(preds))*100)
    # calculate val loss for epoch
    val_losses.append(running_loss / len(val_dataloader))
    # val_losses.append(loss)
    
    # calculate AVG accuracy
    AVG_acc = np.sum(val_acc)/len(val_acc)

    print("[Epoch: %d / %d],  [Train loss: %.4f],  [Train Acc: %.2f],  [Test loss: %.4f],  [Test Acc: %.2f],  [AVG test Acc: %.2f]" \
          %(epoch+1, num_epochs, train_losses[-1],train_acc[-1], val_losses[-1], val_acc[-1],AVG_acc))

SyntaxError: invalid syntax (4236128970.py, line 3)

In [ ]:
import matplotlib.pyplot as plt

def plot_accuracies():
    """ Plot the history of accuracies"""
    plt.plot(train_acc, '-b')
    plt.plot(val_acc, '-r')
    plt.xlabel('epoch')
    plt.ylabel('accuracy')
    plt.legend(['Training', 'Validation'])
    plt.title('Accuracy vs. No. of epochs')
    plt.show()
  
plot_accuracies()

def plot_losses():
    """ Plot the losses in each epoch"""
    plt.plot(train_losses, '-b')
    plt.plot(val_losses, '-r')
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.legend(['Training', 'Validation'])
    plt.title('Loss vs. No. of epochs')
    plt.show()

plot_losses()

In [ ]:
torch.save(model.state_dict(),'./saved_model/VGG16.pth')

In [ ]:
# ls ../input/test-drowsiness/

In [ ]:
test_img = Image.open('../input/test-drowsiness/test2.jpg')
# test_img2 = Image.open('../input/test-drowsiness/test2.jpg')
test_img = test_img.resize((145, 145))
# print(np.array(test_img).shape)
input_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0, 0, 0), (1, 1, 1))
])
test_img = input_transform(test_img)
test_img.unsqueeze_(0)
test_img = test_img.to(device)
output = model(test_img)
_, preds = torch.max(output, 1)
preds  = preds.tolist()
print(preds)
